loading package and seurat object

In [1]:
library(ggplot2)
library(cowplot)
library(dplyr)
library(SCPA)
library(tidyverse)
library(msigdbr)
library(tibble)
library(patchwork)
library(Seurat)
library(rliger)
library(SeuratData)
library(magrittr)
library(SeuratWrappers)
library(viridis)
library(tidyr)
library(ComplexHeatmap)
library(circlize)
library(RColorBrewer)


Attachement du package : ‘dplyr’


Les objets suivants sont masqués depuis ‘package:stats’:

    filter, lag


Les objets suivants sont masqués depuis ‘package:base’:

    intersect, setdiff, setequal, union


Warning message:
“le package ‘SCPA’ a été compilé avec la version R 4.2.2”
This is version 1.5.3 of SCPA
For SCPA tutorials and latest version: https://jackbibby1.github.io/SCPA/
For the SCPA GitHub page: https://github.com/jackbibby1/SCPA
If you use SCPA, please cite: Bibby JA. et al. Cell Rep. 2022


Warning message in system("timedatectl", intern = TRUE):
“l'exécution de la commande 'timedatectl' renvoie un statut 1”
── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ lubridate 1.9.2     ✔ tibble    3.2.1
✔ purrr     1.0.1     ✔ tidyr     1.3.0
✔ readr     2.1.4     
── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyv

In [3]:
custom_colors <- list()

colors_dutch <- c("#16a085", "#2980b9", "#4834d4", "#f39c12", "#c0392b",
                  "#130f40", "#f78fb3", "#65d6ce", "#d1c145", "#2ecc71",
                  "#d35400", "#c44569", '#ff7f50', '#706fd3', '#f9ca24',
                  '#34ace0', '#33d9b2', '#2c2c54', '#be2edd')

colors_spanish <- c(
  '#40407a','#706fd3','#f7f1e3','#34ace0','#33d9b2',
  '#2c2c54','#474787','#aaa69d','#227093','#218c74',
  '#ff5252','#ff793f','#d1ccc0','#ffb142','#ffda79',
  '#b33939','#cd6133','#84817a','#cc8e35','#ccae62'
)

custom_colors$discrete <- c(colors_dutch, colors_spanish)

custom_colors$cell_cycle <- setNames(
  c('#45aaf2', '#f1c40f', '#e74c3c', '#7f8c8d'),
  c('G1',      'S',       'G2M',     '-')
)

In [4]:
options(repr.plot.width = 18, repr.plot.height = 17, repr.plot.pointsize = 24)

In [10]:
scpa_out <- readRDS("/home/adufour/work/rds_storage/microglia_scpa_out.rds")

In [6]:
tb3 <- msigdbr("Mus musculus", category = "H")

In [11]:
microglia <- readRDS("/home/adufour/work/rds_storage/microglia/microglia.rds")

In [12]:
microglia@meta.data$cluster_time <- paste0(microglia@meta.data$PND, "_", microglia@meta.data$seurat_clusters)
microglia_subset <- subset(x = microglia, idents = c("0", "1", "2", "3", "4", "5", "6", "7"))
cell_types <- unique(microglia_subset@meta.data$cluster_time)

In [14]:
for (i in cell_types) {
  scpa_out[[i]] <- scpa_out[[i]] %>%
    select(Pathway, qval) %>%
    magrittr::set_colnames(c("Pathway", paste(i, "qval", sep = "_")))

# For faster analysis with parallel processing, use 'parallel = TRUE' and 'cores = x' arguments
}

In [15]:
scpa_out <- scpa_out %>% 
  reduce(full_join, by = "Pathway") %>% 
  magrittr::set_colnames(gsub(colnames(.), pattern = " ", replacement = "_")) %>%
  select(c("Pathway", grep("_qval", colnames(.)))) %>%
  filter_all(any_vars(. > 2)) %>%
  column_to_rownames("Pathway")

In [25]:
scpa_out <- scpa_out[apply(scpa_out>1.5,1,any),]

In [26]:
scpa_out <- scpa_out[grep("HALLMARK", rownames(scpa_out)),]

In [28]:
rownames(scpa_out) <- sub('HALLMARK_', '', rownames(scpa_out))
rownames(scpa_out) <- sub('_', ' ', rownames(scpa_out))

In [29]:
column_order <- c("P1_0_qval", "P3_0_qval", "P5_0_qval",
                  "P1_1_qval", "P3_1_qval", "P5_1_qval",
                  "P1_2_qval", "P3_2_qval", "P5_2_qval",
                  "P1_3_qval", "P3_3_qval", "P5_3_qval",
                  "P1_4_qval", "P3_4_qval", "P5_4_qval",
                  "P1_5_qval", "P3_5_qval", "P5_5_qval",
                  "P1_6_qval", "P3_6_qval", "P5_6_qval",
                  "P1_7_qval", "P3_7_qval", "P5_7_qval")
mat <- scpa_out[,column_order]
hb <- HeatmapAnnotation(foo = anno_block(gp = gpar(fill = c("#00a085", "#1a7fb6", "#4a2ed0", "#f79c30", "#c13626", "#131140", "#fc8eb3", '#54d5cc')),
                                                   labels = c('0', '1', '2', '3', '4', '5', '6', '7'),
                                         labels_gp = gpar(col = c("black", "white", "white", "black", "white", "white", "black", "black"), fontsize = 48)))
col_fun <- colorRamp2(seq(min(mat), max(mat), length = 100), colorRampPalette(rev(brewer.pal(n = 11, name = "RdYlBu")))(100))
lgd <- list(title = "Q-value", legend_height = unit(6, "cm"), grid_width = unit(1, "cm"), labels_gp = gpar(fontsize = 40), title_gp = gpar(fontsize = 36), title_position = "leftcenter-rot")
split = rep(1:8, each = 3)

In [30]:
options(repr.plot.width=30, repr.plot.height=20) # To set the figure size in Jupyter
pdf('/home/adufour/work/notebook/plots/Microglia/microglia_hallmark_1.5.pdf', width=32,height=20)
hm <- draw(ComplexHeatmap::Heatmap(mat, 
                                   col = col_fun, 
                                   cluster_columns = FALSE, 
                                   column_names_gp=grid::gpar(fontsize=48),
                                   column_split = split, 
                                   column_gap = unit(3, "mm"), 
                                   column_labels = rep(c("P1","P3","P5"),8),
                                   row_names_gp=grid::gpar(fontsize=30),
                                   top_annotation = hb, 
                                   column_title = NULL, 
                                   heatmap_legend_param = lgd), 
           padding = unit(c(2, 2, 2, 160), "mm"),
           heatmap_legend_side = "left")
dev.off()

Warning message:
“The input is a data frame, convert it to a matrix.”


png 
  2